# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm
import scipy.stats as stats

In [2]:
%run -i 'cosim_import.py'

caseH = 18
ict_off = False
ecc_off = False

%run -i 'cosim_setup_andes.py'

%run -i 'cosim_loadsyn.py'

%run -i 'cosim_setup_dispatch.py'

%run -i 'cosim_linktable.py'

# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 3600

rru, rrd, rsfr = 1.0, 1.0, 0.04

# Define functions and constants used in loop
%run -i 'cosim_const.py'

0.post4496.dev0+gc181bac
2.8.0


EVA: ts=18[H], 50000 EVs, Total Q=434.54 MWh
Online 14477, Q=362.27 MWh, SoC=0.7726
Power(MW): Pt=-39.0266, Pc=-39.0266, Pd=-0.0
Ctrl: 0=7897; 1=6580; 
EVA: Load A from Aest.csv.
Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1392 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/ieee39_ev2.xlsx"...
Input file parsed in 0.0646 seconds.
System internal structure set up in 0.0252 seconds.


ANDES total load is: 58.564 p.u.


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0880 seconds.
0: |F(x)| = 7.801234474
1: |F(x)| = 3.497456094
2: |F(x)| = 0.1830765786
3: |F(x)| = 0.0007183022492
4: |F(x)| = 1.659284843e-08
Converged in 5 iterations in 0.0045 seconds.
Power flow results are consistent. Conversion is successful.


Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0056 seconds.
0: |F(x)| = 7.801234474
1: |F(x)| = 3.497456094
2: |F(x)| = 0.1830765786
3: |F(x)| = 0.0007183022492
4: |F(x)| = 1.659284843e-08
Converged in 5 iterations in 0.0074 seconds.
Power flow results are consistent. Conversion is successful.
GSF is consistent.
RTED GB model is loaded.
RTED removed: ['rampu', 'rampd']
RTED limited: sfru, sfrd
RTED: total cost=4889.131


pp cost=2184.5741560778906, gb cost=4889.131241556355


Revise the EV control logic as: C->I, I->D, SOC high to low

In [3]:
# --- Response ---
from ev_ssm import ev_ssm
import numpy as np

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.5, tt_var=0.2, tt_lb=0, tt_ub=1,
             ict_off=True, ecc_off=False)
sse.load_A("Aest.csv")

evd0 = sse.ev.copy()

# quiet some EVs by force
# np.random.seed(sse.config["seed"])
# ict_rid = np.random.randint(low=0,
#                             high=sse.ev.shape[0],
#                             size=3000)

# sse.ev.na.iloc[ict_rid] = sse.ev.nam.iloc[ict_rid]
# sse.ev.lc.iloc[ict_rid] = 1
# sse.g_u()

# initial EV data
evd0 = sse.ev.copy()

sse.g_frc()

EVA: ts=18[H], 10000 EVs, Total Q=87.7 MWh
Online 2925, Q=73.03 MWh, SoC=0.7717
Power(MW): Pt=-7.9463, Pc=-7.9463, Pd=-0.0
Ctrl: 0=1584; 1=1341; 
EVA: Load A from Aest.csv.


[24.629258030047087, 1.8878967125704174]

In [4]:
sse.config["iter_tol"] = 100

In [5]:
sse.run(tf=18 + 5/3600, Pi=20,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

EVA MCS: 100%|██████████| 6/6 [00:01<00:00,  5.06it/s]


In [8]:
sse.data["Pr"]

18.175360044243867

In [7]:
u, v, us, vs, usp, vsp, Pi_cap, error = sse.r_agc(10)

In [ ]:
sse.data["Pr"]

In [ ]:
error

In [ ]:
sse.data

In [ ]:
sse.prdmax

In [ ]:
sse.run(tf=18 + 5/3600, Pi=20,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

sse.run(tf=18 + 60/3600, Pi=25,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

sse.run(tf=18 + 110/3600, Pi=10,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

sse.run(tf=18 + 160/3600, Pi=5,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

# for i in range(10):
#     sse.run(tf=18 + (60+i)/3600, Pi=5-i*0.2,
#             is_updateA=False, is_rstate=True,
#             is_test=False, disable=False)

# sse.run(tf=18 + 100/3600, Pi=0,
#         is_updateA=False, is_rstate=True,
#         is_test=False, disable=False)


In [ ]:
sse.plot_agc()

In [ ]:
sse.tsd.plot(y=['Ptc'])

In [ ]:
sse.tsd

In [ ]:
sse.tsd.plot(y=['Pr', 'Prc'])

In [ ]:
sse.data

In [ ]:
sse.plot_agc()

In [ ]:
sse.data

In [ ]:
sse.run(tf=18 + 5/3600, Pi=10,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

In [ ]:
sse.data

In [ ]:
u, v, us, vs, usp, vsp, error = sse.r_agc(5)

In [ ]:
sse.data

In [ ]:
error

In [ ]:
sse.run(tf=18 + 15/3600, Pi=10,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

In [ ]:
sse.prumax

In [ ]:
sse.prdmax

In [ ]:
u, v, us, vs, usp, vsp, error = sse.r_agc(15)

In [ ]:
error

In [ ]:
sse.data

In [ ]:
sse.plot_agc()

In [ ]:
sse.plot()